**INFORMATION**

```
Panther's ID:       6314061
Student's Name:     Quang Tuan Le
Professor's Name:   Prof. Ron Zacharski
Semester:           Spring 2023 - CAP4770
```

**Intro to Data Mining**

**Project #1 - BACH CHORALES**

#### **1) Prepare Dataset to Process**

In [1]:
# download the zip file from a given link of dataset 
!wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip -O bach.zip 

# unzip the file
!unzip bach.zip 


--2023-03-11 13:58:14--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip’

bach.zip            100%[===================>]  40.78K  --.-KB/s    in 0.01s   

2023-03-11 13:58:14 (3.86 MB/s) - ‘bach.zip’ saved [41761/41761]

Archive:  bach.zip
  inflating: bach.csv                


#### **2) Import Necessary Libraries**

In [ ]:
# import the pandas library 
import pandas as pd 

# import the numpy library
import numpy as np

# import the OneHotEncoder & LabelEncoder for pre-processing data 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# import the model selection libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import RandomizedSearchCV

# import xgboost classifier
from xgboost import XGBClassifier

# import the Bagging & Pasting classifier
from sklearn.ensemble import BaggingClassifier

# import the decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# import the metrics/accuracy library
from sklearn.metrics import accuracy_score


#### **3) Read in and Explore the Data**

In [2]:
import pandas as pd

# read in the data file
bach = pd.read_csv('bach.csv')
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [130]:
# This is just to fix an error in-case when I used another technique, but I don't use it anymore.
# bach = bach.rename(columns={'C':'C_note', 'C#':'C_sharp_note', 'D':'D_note', 'D#':'D_sharp_note', 
#                             'E': 'E_note', 'F':'F_note', 'F#':'F_sharp_note', 'G':'G_note', 'G#':'G_sharp_note', 
#                             'A':'A_note', 'A#':'A_sharp_note', 'B':'B_note'})
# bach

#### **4) Data Analysis**

##### Analyze the dataset

Below code shown that the dataset has several labels only occur once. That's a reason leading to an error when applying XGBoost

In [157]:
# prove that dataset contain labels which only occur once which lead to an error when applying XGBoost
bach[['chord_label']].value_counts()[-10:]

chord_label
D#M            2
C_d7           2
G#M            1
F#d7           1
Abd            1
D#d6           1
EbM7           1
DbM7           1
F_d7           1
Ebd            1
dtype: int64

This code filters and drops the rows of labels which occur once time.

In [3]:
# This code from Professor. Ron Zacharski
import numpy as np

x = bach[['chord_label']].value_counts() 

y = x[x == 1]
z = y.index.get_level_values(0)
for singleton in np.array(z).tolist():
  bach = bach.drop(bach[bach['chord_label']  == singleton].index)

##### Divide the dataset

Divide the dataset into 2 parts including Features and Labels

In [4]:
# features contain every columns except the label
bFeatures = bach.drop('chord_label', axis=1)

# labels will be a chord_label column
bLabels = bach['chord_label']

I have used 'event_number' column as a feature, and the accuracy score is not really affected. But I still prefer to drop it.

In [5]:
# drop the 'event_number' column out of features
bFeatures = bFeatures.drop('event_number', axis=1)

As dataset is not a numerical data, applying OneHotEncoder for the features

In [6]:
# One hot encode the features using sklearn's OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
bFeatures = enc.fit_transform(bFeatures)
bFeatures

<5657x105 sparse matrix of type '<class 'numpy.float64'>'
	with 84855 stored elements in Compressed Sparse Row format>

So the label values are strings. However, XGBoost requires the values to be integers starting at 0. We can use sklearn's LabelEncoder to do the conversion:

In [7]:
# LabelEncoder for Bach Labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bLabels = le.fit_transform(bLabels)

Now we can divide the data into training and testing ...

In [8]:
# divide the data into training and testing sets
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42, stratify=bLabels)
bach_train_features 

<4525x105 sparse matrix of type '<class 'numpy.float64'>'
	with 67875 stored elements in Compressed Sparse Row format>

#### **5) XGBOOST Classifier**

**XGBOOST**

I will start with basic XGBoost Classifier with default hyperparameters.

In [16]:
# Create parameters for initialize XGBoost Classifier 
from xgboost import XGBClassifier
params = {"n_estimators": 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}

model = XGBClassifier(**params)
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              predictor='gpu_predictor', random_state=None, ...)

In [17]:
# XGBoost model fit the training set
model.fit(bach_train_features, bach_train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor='gpu_predictor', ...)

In [18]:
# Get predictions and accuracy score
from sklearn.metrics import accuracy_score
model_predictions = model.predict(bach_test_features)
accuracy_score(bach_test_labels, model_predictions)

0.7659010600706714

##### **XGBoost Finding Process**

Find the best hyperparameters as I could. I use GridSearchCV to set several different parameters, and combine with StratifiedKFold & RandomizedSearchCV to find the best parameters.

In [15]:
# Set up param grid
from sklearn.model_selection import GridSearchCV
param_grid = [
    {
        'n_estimators': [50, 100, 150, 200],
        'max_depth': [2, 4, 6, 8]
    }
]

In [19]:
# Set up StratifiedKFold and apply RandomizedSearchCV to XGBoost model
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import RandomizedSearchCV
param_comb = 5
folds = 5
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1001)
model_random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb, n_jobs=-1,
                                         cv=skf.split(bach_train_features, bach_train_labels), verbose=3)

Starting training ... The process takes ~ 6 minutes 28 seconds

In [20]:
%%time
model_grid_search = model_random_search.fit(bach_train_features, bach_train_labels)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


CPU times: user 5.69 s, sys: 384 ms, total: 6.08 s
Wall time: 6min 28s


In [21]:
# get the best param among params
model_random_search.best_params_

{'n_estimators': 50, 'max_depth': 6}

In [22]:
# make predictions
model_random_search_predictions = model_random_search.best_estimator_.predict(bach_test_features)

In [23]:
# get accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, model_random_search_predictions)

0.7667844522968198

##### **The best accuracy score I can get when applying XGBoost is 76.68%**

#### **6) Decision Tree Classifier**

In these block, I am going to use Decision Tree entropy to train the dataset.

**Decision Tree**

In [167]:
# Initial Decision Tree Classifier and fit the training set
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(criterion='entropy')
tree_clf.fit(bach_train_features, bach_train_labels)

DecisionTreeClassifier(criterion='entropy')

In [168]:
# get predictions and accuracy score
tree_clf_predictions = tree_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, tree_clf_predictions)

0.7508833922261484

**For basic decision tree entropy, the accuracy score is 75.09%**

#### **7) Bagging & Pasting Classifier**

The following steps are only about using BaggingClassifier with different versions of its based on HyperParameters passed. I would try to figure out which the best accuracy score I could. 

Let's check it out!

##### Basic Parameter for each types of Bagging Classifier

**Initial Decision Trees Entropy**

In [169]:
# Initial decision tree to apply to Bagging Classifier
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')

**Bagging Classifier**

In [170]:
# Bagging Classifier with below parameters
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, bootstrap=True, n_jobs=-1)
bagging_clf.fit(bach_train_features, bach_train_labels)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy'),
                  max_samples=100, n_estimators=20, n_jobs=-1)

In [171]:
# Get predictions and accuracy score
bagging_predictions = bagging_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, bagging_predictions)

0.661660777385159

**Pasting Classifier**

In [172]:
# Pasting Classifier with below parameters
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100,
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(bach_train_features, bach_train_labels)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(criterion='entropy'),
                  max_samples=100, n_estimators=20, n_jobs=-1)

In [173]:
# Get predictions and accuracy score
pasting_predictions = pasting_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, pasting_predictions)

0.6704946996466431

**Random Subspaces**

In [174]:
# Random Subspaces Classifier with below parameters
subspace_clf = BaggingClassifier(clf, n_estimators=50, max_features=7, 
                                 bootstrap=True, n_jobs=-1)
subspace_clf.fit(bach_train_features, bach_train_labels)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy'),
                  max_features=7, n_estimators=50, n_jobs=-1)

In [175]:
# Get predictions and accuracy score
subspace_predictions = subspace_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, subspace_predictions)

0.5468197879858657

**Random Patches**

In [176]:
# Random Patches with below parameters
patches_clf = BaggingClassifier(clf, n_estimators=100, max_features=7,
                                max_samples=100, bootstrap=False, n_jobs=-1)
patches_clf.fit(bach_train_features, bach_train_labels)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(criterion='entropy'),
                  max_features=7, max_samples=100, n_estimators=100, n_jobs=-1)

In [177]:
# Get predictions and accuracy score
patches_predictions = patches_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, patches_predictions)

0.5053003533568905

The accuracy scores above are all less than XGBoost, so I am going to use mixing parameters. 

**Try New Thing**

In [178]:
best_bagging_clf = BaggingClassifier(clf, n_estimators=200, max_samples=0.5, max_features=1.0, 
                                     bootstrap=True, bootstrap_features=False)
best_bagging_clf.fit(bach_train_features, bach_train_labels)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy'),
                  max_samples=0.5, n_estimators=200)

In [179]:
best_bagging_predictions = best_bagging_clf.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, best_bagging_predictions)

0.7685512367491166

I got new peak of the accuracy score which is 76.86%

**I got the best accuracy so far. But, I will use GridSearchCV to find the best hyperparameters**


In [180]:
# Set up param grid and fitting bagging classifier to the training set
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

# Set up params
param_grid_version = {
    'n_estimators': [10, 50, 100, 200, 500],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False]
}

# Init Bagging Classifier with Decision Tree
bagging_clf_version = BaggingClassifier(estimator=DecisionTreeClassifier())

# Run random search
random_search_version = RandomizedSearchCV(estimator=bagging_clf_version, param_distributions=param_grid_version, 
                                           n_iter=20, cv=5, random_state=42)

# Fitting model to the training set
random_search_version.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=BaggingClassifier(estimator=DecisionTreeClassifier()),
                   n_iter=20,
                   param_distributions={'bootstrap': [True, False],
                                        'bootstrap_features': [True, False],
                                        'max_features': [0.5, 0.8, 1.0],
                                        'max_samples': [0.5, 0.8, 1.0],
                                        'n_estimators': [10, 50, 100, 200,
                                                         500]},
                   random_state=42)

Run Time: 10 minutes

In [181]:
# Get the best param
random_search_version.best_params_

{'n_estimators': 500,
 'max_samples': 0.8,
 'max_features': 0.8,
 'bootstrap_features': True,
 'bootstrap': True}

In [182]:
# Predict the test features
rsv_predictions = random_search_version.best_estimator_.predict(bach_test_features)

In [183]:
# Get accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, rsv_predictions)

0.7747349823321554

##### **Here is the new peak of accuracy score which is 77.5%**

##### **Continuing finding the best params with Bagging Classifier**

**ANOTHER FINDING**

Run time: 15 minutes

In [184]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

param_grid_v2 = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.65, 0.75, 0.95],
    'max_features': [0.5, 0.75, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
}

dt = DecisionTreeClassifier()

bagging_clf_v2 = BaggingClassifier(estimator=dt)

grid_search_v2 = GridSearchCV(bagging_clf_v2, param_grid=param_grid_v2, cv=5)
grid_search_v2.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier()),
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 0.75, 1.0],
                         'max_samples': [0.65, 0.75, 0.95],
                         'n_estimators': [10, 50, 100]})

In [185]:
grid_search_v2.best_params_

{'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 0.5,
 'max_samples': 0.95,
 'n_estimators': 100}

In [186]:
gsv2_predictions = grid_search_v2.best_estimator_.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, gsv2_predictions)

0.769434628975265

**ANOTHER ONES**

Run time: 22 minutes

In [9]:
# Same process 
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Set up params
param_grid_v3 = {
    'n_estimators': [100, 200, 500],
    'max_samples': [0.6, 0.75, 0.9],
    'max_features': [0.5, 0.75, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
}

# Create new decision tree
dt = DecisionTreeClassifier(max_depth=5)

# Init BaggingClassifier
bagging_clf_v3 = BaggingClassifier(estimator=dt)

# Search and fit model to training set
grid_search_v3 = GridSearchCV(bagging_clf_v3, param_grid=param_grid_v3, cv=5)
grid_search_v3.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5)),
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 0.75, 1.0],
                         'max_samples': [0.6, 0.75, 0.9],
                         'n_estimators': [100, 200, 500]})

Next, get best params and make predictions

In [10]:
grid_search_v3.best_params_

{'bootstrap': True,
 'bootstrap_features': True,
 'max_features': 0.5,
 'max_samples': 0.9,
 'n_estimators': 100}

In [11]:
gsv3_predictions = grid_search_v3.best_estimator_.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, gsv3_predictions)

0.6819787985865724

**ONE MORE**

Run time: 9 minutes

In [12]:
# Same process
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Set up params
param_grid_v4 = {
    'n_estimators': [150, 250, 300],
    'max_samples': [0.5, 0.75, 1.0],
    'max_features': [0.5, 0.75, 1.0],
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
}

# New decision tree
dt = DecisionTreeClassifier(max_depth=10)

# Init Bagging Classifier
bagging_clf_v4 = BaggingClassifier(estimator=dt)

# Search and fit model to the training set
random_search_v4 = RandomizedSearchCV(estimator=bagging_clf_v4, param_distributions=param_grid_v4, 
                                           n_iter=20, cv=5, random_state=42)

random_search_v4.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=10)),
                   n_iter=20,
                   param_distributions={'bootstrap': [True, False],
                                        'bootstrap_features': [True, False],
                                        'max_features': [0.5, 0.75, 1.0],
                                        'max_samples': [0.5, 0.75, 1.0],
                                        'n_estimators': [150, 250, 300]},
                   random_state=42)

Next, find the best params and make predictions

In [13]:
random_search_v4.best_params_

{'n_estimators': 250,
 'max_samples': 0.75,
 'max_features': 0.75,
 'bootstrap_features': True,
 'bootstrap': False}

In [14]:
rsv4_predictions = random_search_v4.best_estimator_.predict(bach_test_features)
from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, rsv4_predictions)

0.7641342756183745

#### **8) Summary**

As I follow several techniques I have learned as my Table of Contents shown such as using Decision Tree entropy, Pre-processing DataSet before using BaggingClassifier and XGBoost. I got the accuracy score as best as I could for the Bach Chorales project.

I also marked the milestone to my Table of Contents.

##### **My best accuracy score is 77.5%.**